# Imports

In [15]:
import pandas as pd
import tensorflow as tf
import tensorflow.keras.layers as Layer
import numpy as np
# import keras_tuner as kt
import tensorflow.keras.layers.experimental.preprocessing as preprocessing
from DatasetFormatting import *

# Preparing Training Dataset

In [16]:
np.random.seed(1)
tf.random.set_seed(1)

movie_df = df
movie_df.drop(['vote_count'], axis = 1)

# saves genres keys/values if needed for lookup later
# keys, genres = pd.factorize(movie_df['genres'])

# print(genres.shape)
# movie_df['genres'] = pd.factorize(movie_df['genres'])[0]

data_count = movie_df.shape[0]
training_data_count = int(data_count * 0.75) + 1
test_data_count = data_count - training_data_count

y_train = np.array(movie_df['vote_average'].head(training_data_count))
x_train = np.array(movie_df['final_form'].head(training_data_count))

y_test = np.array(movie_df['vote_average'].tail(test_data_count))
x_test = np.array(movie_df[['final_form']].tail(test_data_count))

# print(x_train)
# print(x_train.shape)


# print(x_test)
# print(x_test.shape)

# Text Vectorization For Features

In [17]:
VOCAB_SIZE = 201277
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE)

features = np.array(df['final_form'])


encoder.adapt(features)
vocab = np.array(encoder.get_vocabulary())
vocab

2021-11-05 23:46:52.938838: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


array(['', '[UNK]', 'drama', ..., 'aamnashariff', 'aamirbashir',
       'aadukalamnaren'], dtype='<U30')

In [18]:
example = encoder(movie_df['final_form'].to_numpy()[0]).numpy()
print(movie_df['final_form'].to_numpy()[0])
example

DakotaFanning JosePabloCantillo Horror


array([ 325, 3016,    5])

# Recurrent Neural Network

In [19]:



model_RNN = tf.keras.Sequential([
 encoder,
    Layer.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=100,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    Layer.SimpleRNN(100),
    Layer.Dense(100, activation='relu'),
    Layer.Dense(1)
])

model_RNN.compile(loss=tf.keras.losses.MeanSquaredError(),
              optimizer=tf.keras.optimizers.Adam(1e-3),
              metrics=['accuracy'])

model_RNN.summary()

# Train the model.
model_RNN.fit(
    x_train,
    y_train, 
    batch_size=512,
    epochs=5)

# Evaluate
score = model_RNN.evaluate(x_train, y_train)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization_2 (TextVe (None, None)              0         
_________________________________________________________________
embedding (Embedding)        (None, None, 100)         4126500   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 100)               20100     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 4,156,801
Trainable params: 4,156,801
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5


2021-11-05 23:46:53.789957: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


34/34 [==============================] - 3s 70ms/step - loss: 14.4052 - accuracy: 5.8862e-05
Epoch 2/5
34/34 [==============================] - 2s 65ms/step - loss: 1.8268 - accuracy: 0.0000e+00
Epoch 3/5
34/34 [==============================] - 2s 70ms/step - loss: 1.0543 - accuracy: 0.0000e+00
Epoch 4/5
34/34 [==============================] - 2s 62ms/step - loss: 0.6089 - accuracy: 0.0000e+00
Epoch 5/5
  5/531 [..............................] - ETA: 7s - loss: 0.4389 - accuracy: 0.0000e+00  

2021-11-05 23:47:05.526234: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


531/531 [==============================] - 7s 12ms/step - loss: 0.1850 - accuracy: 0.0000e+00
Test loss: 0.1850317120552063
Test accuracy: 0.0


In [20]:
sample_text = ('action')
predictions = model_RNN.predict(np.array([sample_text]))
print(predictions[0])




[2.4510148]


2021-11-05 23:47:12.119678: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [21]:
# input = tf.keras.layers.Input(shape=(1,))
# genre_embedding = tf.keras.layers.Embedding(
#         input_dim=len(genre_encoder.get_vocabulary()) + 1,
#         output_dim=100)(input)


# simple_rnn = tf.keras.layers.SimpleRNN(100)(genre_embedding)
# hidden1 = tf.keras.layers.Dense(100, activation='relu')(simple_rnn)
# output = tf.keras.layers.Dense(1)(hidden1)

# model_RNN = keras.Model(inputs=input, outputs=output)
